In [2]:
! pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 58.3 MB/s eta 0:00:00


In [3]:
import numpy as np
from pulp import *

In [5]:
filename = "Tsp58_2.txt"
matrix = []
with open(filename, 'r') as file:
    for line in file:
        row = list(map(int, line.strip().split()))
        matrix.append(row)

m58 = np.zeros([58,58])
for i in range(0,len(m58)-1):
    for j in range(0,len(matrix[i])):
        m58[i][i+j+1]=matrix[i][j]

for i in range(0,len(m58)):
  for j in range(0,len(m58)):
    if i==j:
        m58[i][j]=0
    elif i < j:
        m58[j][i]=m58[i][j]
print(m58)
matriz_distancias=m58[:5,:5]
print(matriz_distancias)

size = 5

departure = [str(i) for i in range(size)]
arrival = [str(i) for i in range(size)]

distance = {}

for dep in departure:
    distance[dep] = {}
    for arr in arrival:
        cost = matriz_distancias[int(dep)][int(arr)]
        distance[dep][arr] = cost

[[   0. 2635. 2713. ... 3870. 1417.  739.]
 [2635.    0.  314. ... 2072. 1196. 1517.]
 [2713.  314.    0. ... 1882. 2699. 1557.]
 ...
 [3870. 2072. 1882. ...    0. 2328. 2986.]
 [1417. 1196. 2699. ... 2328.    0.  962.]
 [ 739. 1517. 1557. ... 2986.  962.    0.]]
[[   0. 2635. 2713. 2437. 1600.]
 [2635.    0.  314. 2636.  666.]
 [2713.  314.    0. 2730.  706.]
 [2437. 2636. 2730.    0. 2824.]
 [1600.  666.  706. 2824.    0.]]


In [7]:
lp_problem = LpProblem("Traveling Salesman Problem", LpMinimize)

routes = [ (d, a) for d in departure for a in arrival ]

vars = LpVariable.dicts("best route", (departure, arrival), 0, 1, LpInteger)
vars_f = LpVariable.dicts('Flow', (departure, arrival), 0, None, LpInteger)
n = len(departure)-1

lp_problem += (lpSum([vars[o][d] * distance[o][d] for (o, d) in routes]),"Soma_das_distancias_percorridas", )

for a in arrival:
    lp_problem += ( lpSum([vars[d][a] for d in departure]) == 1, f"Sum_Arrivel_{a}",)

for d in departure:
    lp_problem += ( lpSum([vars[d][a] for a in arrival]) == 1, f"Sum_departure_{d}",)

lp_problem += (
        lpSum([vars[a][a] for a in arrival]) == 0, f"Sum_{a,a}",)

for d in departure:
        for a in arrival:
                lp_problem += (
                        vars_f[d][a] <= n*vars[d][a]
                        )

for o in ['1','2','3','4']:
        lp_problem += (
                lpSum([vars_f[a][o]-vars_f[o][a] for a in arrival]) == 1,
                f"Flow_balance_{o}",
                )

In [9]:
lp_problem.solve()
print("Status:", LpStatus[lp_problem.status])

Status: Optimal


In [11]:
if lp_problem.status == LpStatusOptimal:
    print("Solução ótima encontrada:")
    for var in lp_problem.variables():
        print(var.name, "=", var.varValue)
else:
    print("Não foi possível encontrar uma solução ótima.")

Solução ótima encontrada:
Flow_0_0 = 0.0
Flow_0_1 = 0.0
Flow_0_2 = 0.0
Flow_0_3 = 0.0
Flow_0_4 = 4.0
Flow_1_0 = 0.0
Flow_1_1 = 0.0
Flow_1_2 = 0.0
Flow_1_3 = 1.0
Flow_1_4 = 0.0
Flow_2_0 = 0.0
Flow_2_1 = 2.0
Flow_2_2 = 0.0
Flow_2_3 = 0.0
Flow_2_4 = 0.0
Flow_3_0 = 0.0
Flow_3_1 = 0.0
Flow_3_2 = 0.0
Flow_3_3 = 0.0
Flow_3_4 = 0.0
Flow_4_0 = 0.0
Flow_4_1 = 0.0
Flow_4_2 = 3.0
Flow_4_3 = 0.0
Flow_4_4 = 0.0
best_route_0_0 = 0.0
best_route_0_1 = 0.0
best_route_0_2 = 0.0
best_route_0_3 = 0.0
best_route_0_4 = 1.0
best_route_1_0 = 0.0
best_route_1_1 = 0.0
best_route_1_2 = 0.0
best_route_1_3 = 1.0
best_route_1_4 = 0.0
best_route_2_0 = 0.0
best_route_2_1 = 1.0
best_route_2_2 = 0.0
best_route_2_3 = 0.0
best_route_2_4 = 0.0
best_route_3_0 = 1.0
best_route_3_1 = 0.0
best_route_3_2 = 0.0
best_route_3_3 = 0.0
best_route_3_4 = 0.0
best_route_4_0 = 0.0
best_route_4_1 = 0.0
best_route_4_2 = 1.0
best_route_4_3 = 0.0
best_route_4_4 = 0.0


In [13]:
print("Custo total:", pulp.value(lp_problem.objective))

Custo total: 7693.0
